In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import os
import scipy.signal

from spykshrk.realtime.decoder_process import PointProcessDecoder

from spykshrk.realtime.simulator import nspike_data

from spykshrk.franklab.pp_decoder.util import gaussian, normal2D, apply_no_anim_boundary, simplify_pos_pandas
from spykshrk.franklab.pp_decoder.pp_clusterless import OfflinePPDecoder
from spykshrk.franklab.pp_decoder.data_containers import EncodeSettings, DecodeSettings, SpikeObservation, \
                                                         LinearPosition, StimLockout, Posteriors
from spykshrk.franklab.pp_decoder.visualization import DecodeVisualizer

from spykshrk.franklab.pp_decoder.decode_error import LinearDecodeError

pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)
#pd.set_option('display.width', 180)


idx = pd.IndexSlice


In [2]:
%%time
# Load merged rec HDF store based on config

config_file = '/opt/data36/daliu/realtime/spykshrk/dec_60uv_300samp/bond.config.json'
#config_file = '/home/daliu/Src/spykshrk_realtime/config/bond_single.json'

config = json.load(open(config_file, 'r'))
day = config['simulator']['nspike_animal_info']['days'][0]
epoch = config['simulator']['nspike_animal_info']['epochs'][0]

# Main hdf5 data source file name
hdf_file = os.path.join(config['files']['output_dir'],
                        '{}.rec_merged.h5'.format(config['files']['prefix']))

# Extract just encode and decode settings from config
encode_settings = EncodeSettings(config)
decode_settings = DecodeSettings(config)

# Open data file
store = pd.HDFStore(hdf_file, mode='r')

# Encapsulate Spike Observation panda table in container
observ_obj = SpikeObservation.create_default(store['rec_3'], day=day, epoch=epoch)

# Grab stimulation lockout times
stim_lockout = StimLockout.create_default(store['rec_11'])


# Grab animal linearized real position
nspike_anim = nspike_data.AnimalInfo(**config['simulator']['nspike_animal_info'])
pos = nspike_data.PosMatDataStream(nspike_anim)
pos_data = pos.data

# Encapsulate linear position
lin_obj = LinearPosition.from_nspike_posmat(pos_data, encode_settings)

In [3]:
%%time
# Run PP decoding algorithm

time_bin_size = 300

decoder = OfflinePPDecoder(lin_obj=lin_obj, observ_obj=observ_obj,
                           encode_settings=encode_settings, decode_settings=decode_settings, 
                           which_trans_mat='learned')

posteriors = decoder.run_decoder()

In [4]:
%%time
# Run spykshrk.realtime version of point process decoding

# Create and setup online point process decoder
pp_decoder = PointProcessDecoder(pos_range=[config['encoder']['position']['lower'],
                                            config['encoder']['position']['upper']],
                                 pos_bins=config['encoder']['position']['bins'],
                                 time_bin_size=config['pp_decoder']['bin_size'],
                                 arm_coor=config['encoder']['position']['arm_pos'],
                                 uniform_gain=config['pp_decoder']['trans_mat_uniform_gain'])

pp_decoder.select_ntrodes(config['simulator']['nspike_animal_info']['tetrodes'])

observ_obj.update_observations_bins(time_bin_size)

num_time_bins = observ_obj['dec_bin'].max()

# Group by bin
groups = observ_obj.groupby('dec_bin')

last_bin_id = 0
bin_timestamps = []
spykshrk_posteriors = np.zeros([num_time_bins+1, config['encoder']['position']['bins']])

for bin_id, spikes_in_bin in groups:
    bin_timestamps.append(spikes_in_bin['dec_bin_start'].iloc[0])
    if last_bin_id <= bin_id - 1:
        # increment bins with no spikes
        for bin_no_spk_id in range(last_bin_id + 1, bin_id):
            bin_timestamps.append(bin_timestamps[-1] + time_bin_size)
            post = pp_decoder.increment_no_spike_bin()
            spykshrk_posteriors[bin_no_spk_id, :] = post
        
    # Add 
    for ntrode_id, dec in zip(spikes_in_bin.loc[:, 'ntrode_id'].values, 
                   spikes_in_bin.loc[:, 'x000': 'x{:03d}'.
                                     format(config['encoder']['position']['bins']-1)].values):
        pp_decoder.add_observation(ntrode_id, dec)
        
    post = pp_decoder.increment_bin()
    spykshrk_posteriors[bin_id, :] = post
    last_bin_id = bin_id
    
spykshrk_posteriors = Posteriors.from_numpy(spykshrk_posteriors, day=day, epoch=epoch, 
                                            timestamps=np.array(bin_timestamps),
                                            times=np.array(bin_timestamps)/30000, columns=
                                           ['x{:03d}'.format(pos_ind) for pos_ind in 
                                            range(config['encoder']['position']['bins'])], 
                                            encode_settings=encode_settings)

In [8]:
spykshrk_dec_pos = spykshrk_posteriors.get_distribution_view().idxmax(axis=1).apply(lambda x: int(x[1:])).to_frame()
spykshrk_dec_pos.columns = ['est_pos']

resamp_lin_obj = lin_obj.get_resampled(time_bin_size).get_pd_no_multiindex()

decerr_obj = LinearDecodeError()

dec_error = decerr_obj.calc_error_table(resamp_lin_obj, spykshrk_dec_pos,
                                       encode_settings.arm_coordinates, 2)

print(dec_error.loc[:, idx[:, 'abs_error']].median())
print(dec_error.loc[:, idx[:, 'abs_error']].mean())

In [14]:
error_bars = dec_error.loc[:,idx[:, ['plt_error_up', 'plt_error_down']]]. \
        reindex(columns=pd.MultiIndex.from_product([['center','left','right'],['plt_error_up','plt_error_down']]))

error_bars = np.reshape(error_bars.values, [len(error_bars),3,2])
error_bars = error_bars.transpose(1,2,0)

dec_error.loc[:, idx[:, 'real_pos']].plot(figsize=[100,10], style='o', yerr=error_bars)

plt.show()